Network overview using Xatu sentry node observations on Ethereum mainnet.

In [ ]:
import polars as pl
import plotly.express as px

from loaders import load_parquet, display_sql

target_date = None  # Set via papermill, or auto-detect from manifest

In [ ]:
display_sql("xatu_client_connectivity", target_date)

In [ ]:
df = pl.from_pandas(load_parquet("xatu_client_connectivity", target_date))

## Total unique peers

Number of unique peers observed by Xatu sentry nodes throughout the day.

In [ ]:
df_unique = (
    df
    .group_by("hour_bucket")
    .agg(unique_peers=pl.col("peer_id").n_unique())
    .sort("hour_bucket")
)

fig = px.line(
    df_unique.to_pandas(),
    x="hour_bucket",
    y="unique_peers",
)
fig.update_layout(
    xaxis_title=None,
    yaxis_title="Unique peers",
    height=400,
)
fig.show()

## Client distribution

Distribution of unique peers by client implementation over time. Each peer is assigned to a single client based on their most recently observed agent string.

In [ ]:
df_clients = (
    df
    .filter(pl.col("client_name").is_not_null() & (pl.col("client_name") != ""))
    .sort(["hour_bucket", "peer_id", "client_name"], descending=[False, False, True])
    .unique(subset=["hour_bucket", "peer_id"], keep="first")
    .group_by(["hour_bucket", "client_name"])
    .agg(peers=pl.len())
    .sort(["hour_bucket", "peers"])
)

fig = px.area(
    df_clients.to_pandas(),
    x="hour_bucket",
    y="peers",
    color="client_name",
)
fig.update_layout(
    xaxis_title=None,
    yaxis_title="Peers",
    legend_title="Client",
    height=500,
)
fig.show()

## Connections per Xatu node

Number of unique peers connected to each Xatu sentry node over time.

In [ ]:
df_xatu = (
    df
    .group_by(["hour_bucket", "local_name"])
    .agg(peers=pl.col("peer_id").n_unique())
    .sort("hour_bucket")
    .with_columns(
        pl.col("local_name").str.replace("ethpandaops/mainnet/", "")
    )
)

fig = px.line(
    df_xatu.to_pandas(),
    x="hour_bucket",
    y="peers",
    color="local_name",
)
fig.update_layout(
    xaxis_title=None,
    yaxis_title="Connected peers",
    legend=dict(
        title="Xatu node",
        orientation="h",
        yanchor="top",
        y=-0.2,
        xanchor="center",
        x=0.5,
    ),
    height=500,
)
fig.show()

## Transport protocol distribution

Distribution of connections by IP protocol (IPv4/IPv6) and transport protocol (TCP/QUIC) combinations.

In [ ]:
df_transport = (
    df
    .group_by(["hour_bucket", "peer_id", "protocol"])
    .agg(
        all_transports=pl.col("transport_protocol").unique().sort().str.join(" & ")
    )
    .with_columns(
        protocol_combos=pl.col("protocol") + " + (" + pl.col("all_transports") + ")"
    )
    .group_by(["hour_bucket", "protocol_combos"])
    .agg(peers=pl.len())
    .sort("hour_bucket")
)

fig = px.line(
    df_transport.to_pandas(),
    x="hour_bucket",
    y="peers",
    color="protocol_combos",
)
fig.update_layout(
    xaxis_title=None,
    yaxis_title="Connected peers",
    legend_title="Protocol",
    height=500,
)
fig.show()

## Port popularity

Most commonly used ports by connected peers. Standard Ethereum P2P port is 30303.

In [ ]:
df_ports = (
    df
    .unique(subset=["peer_id", "port"])
    .group_by("port")
    .agg(peers=pl.len())
    .with_columns(pl.col("port").cast(pl.Utf8))
    .sort("peers", descending=True)
    .head(20)
)

fig = px.bar(
    df_ports.to_pandas(),
    x="port",
    y="peers",
)
fig.update_xaxes(type="category")
fig.update_layout(
    xaxis_title="Port",
    yaxis_title="Peers",
    height=400,
)
fig.show()